In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [2]:
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pio.renderers.default='browser'  

def get_candlestick_chart(df: pd.DataFrame):  

    # Make Plots
    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=(f"Stock Price", "Volume", "RangePerVolume")
    )

    # Remove holidays
    dt_all = pd.date_range(start=df.index[0], end=df.index[-1])
    dt_breaks = [ d for d in dt_all.strftime("%Y-%m-%d").to_list() if not d in df.index.astype('str').to_list() ]
    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

    # Candle Sticks Box
    fig.add_trace(
        go.Candlestick(
            x = df.index,
            open = df['시가'], 
            high = df['고가'],
            low = df['저가'],
            close = df['종가'],
            name = 'Candlestick chart'
        ),
        row = 1,
        col = 1
    ) 

    # Moving Averages
    ma_list = [5, 20, 60, 120]
    color_list = ["green", "red", "orange", "purple"]

    for i, c in zip(ma_list, color_list):
        df[f"MA{i}"] = df['종가'].rolling(window=i).mean()
        fig.add_trace(
                go.Scatter(
                x=df.index,
                y=df[f"MA{i}"],
                opacity=0.2,
                line=dict(color=f"{c}", width=2),
                name=f"MA {i}"
            ),
            row=1,
            col=1
        )

    # Volume Box
    fig.add_trace(
        go.Bar(x = df.index, y = df["거래량"], name="Volume"),
        row=2,
        col=1
    )

    # Range Per Volume Box
    fig.add_trace(
        go.Bar(x = df.index, y = df["RangePerVolume"], name="RangePerVolume"),
        row=3,
        col=1
    )

    fig['layout']['xaxis2']['title'] = "Date"
    fig['layout']['yaxis']['title'] = "Price"
    fig['layout']['yaxis2']['title'] = "Volume"
    fig['layout']['yaxis3']['title'] = "RangePerVolume"

    # Remove range slider
    fig.update_xaxes(
        rangeslider_visible = False,
        ) 

    return fig

In [3]:
# tickers = stock.get_market_ticker_list(market="KOSPI")
# ticker_name = {stock.get_market_ticker_name(ticker):ticker for ticker in tickers}
# with open("KOSDAQ.json", "w") as of:
#     json.dump(ticker_name, of)

In [4]:
with open("KOSPI.json", "r") as f:
    ticker_dict = json.load(f)

with open("KOSDAQ.json", "r") as f:
    ticker_dict.update(json.load(f))


In [11]:
ticker = "삼성전자"
df = stock.get_market_ohlcv("20220102", datetime.today().strftime("%Y%m%d"), ticker_dict[ticker])

In [12]:
df["Range"] = df["고가"] - df["저가"]
df["RangePerVolume"] = (df["Range"] / df["거래량"]) * 100_000

In [13]:
rpv_decile =df["RangePerVolume"].quantile(np.linspace(0.1,1.0,num=10))
vol_decile =df["거래량"].quantile(np.linspace(0.1,1.0,num=10))

In [14]:
# df.index.name = "Date"
# df = df.set_axis(["Open", "High", "Low", "Close", "Volume", "Change(%)", "Range", "RangePerVolume"], axis=1)

In [15]:
fig = get_candlestick_chart(df)


In [16]:
fig.add_trace(
    go.Scatter(
        name="Extreme",
         x = [ df.index.min(), df.index.max()  ],
         y = [rpv_decile.iloc[-2], rpv_decile.iloc[-2]],
        mode = "lines",
        marker = dict(color="blue"),
        opacity=0.2
    ), row=3, col=1
)
fig.add_trace(
    go.Scatter(
        name="Extreme",
         x = [ df.index.min(), df.index.max()  ],
         y = [vol_decile.iloc[-2], vol_decile.iloc[-2]],
        mode = "lines",
        marker = dict(color="blue"),
        opacity=0.2
    ), row=2, col=1
)